In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import random
import ipywidgets as widget
from ipycanvas import Canvas, hold_canvas
from time import sleep

In [5]:
class my_pdf(st.rv_continuous):
    def setFunction(self,f):
        self.f = f
    def getFunction(self):
        return f
    def _pdf(self,v):
        return self.f(v)

In [6]:
class particle:
    
    def rotMat(alpha, x):
        return np.array([[np.cos(alpha), -np.sin(alpha)], [np.sin(alpha)], np.cos(alpha)]).dot(x)
    
    def pdf(v):
        return n*(m/(2*pi*k*T))**(3/2)*4*pi*v**2*np.exp(-m*v**2/(2*k*T))
    
    def create(n):
        y = np.array([])
        m = my_pdf(a=0, b=10000)
        m.setFunction(particle.pdf)
        for _ in range(n):
            k = m.rvs()
            y=np.append(y,particle(m,3,3,random.randint(-50,50),random.randint(-50,50),2))
        return y
    
    def collide(n):
        x = [i.getX() for i in n]
        y = [i.getY() for i in n]
        
        for i in range(len(x)):
            for j in range(len(x)-(i+1)):
                print((i,j+i+1))
                print(x[i]-x[j+i+1])
                dx = x[i]-x[j+i+1]
                dy = y[i]-y[j+i+1]
                
                if np.sqrt(dx**2+dy**2)<=n[i].getR()+n[j+i+1].getR():
                    bump(n[i], n[j+i+1], dx,dy)
            
    def bump(x,y,dx,dy):
        v1 = np.array(x.getVelocity())
        v2 = np.array(y.getVelocity())
        v1s = rotMat(np.arctan2(dy,dx),v1)
        v2s = rotMat(np.arctan2(dy,dx),v2)
        
        
        
    
    def __init__(self, m, x,y, vx, vy,r):
        self.m = m
        self.vx = vx
        self.vy = vy
        self.x = x
        self.y = y
        self.r = r
        
    def getPosition(self):
        return (self.x,self.y)
    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getVelocity(self):
        return (self.vx,self.vy)
    def getVx(self):
        return self.vx
    def getVy(self):
        return self.vy
    def getR(self):
        return self.r
    
    def move(self, dt):
        self.x = self.vx * dt + self.x
        self.y = self.vy * dt + self.y
    
    def collisionTest(self, width, height):
        if self.x <= self.r:
            self.vx = -self.vx
        elif self.x >= width - self.r:
            self.vx = -self.vx
            
        if self.y <= self.r: 
            self.vy = -self.vy
        elif self.y >= height - self.r:
            self.vy = -self.vy

In [7]:
pi = np.pi
k = 1.380649 * 10**-23 #m**2 kg s**-2 K**-1

In [8]:
m = 4.002602**1.66*10**(-27) #kg
n = 1 #mol
T = 273.15 #K
width = 400
height = 200

In [9]:
canvas = Canvas(width=width, height=height)

p = particle.create(1000)
steps_number = 100
dt = 0.02
display(canvas)


for _ in range(steps_number):
    with hold_canvas():
        canvas.clear()
        for i in p:
            canvas.fill_circle(i.getX(), i.getY(), i.getR())
            i.move(dt)
            i.collisionTest(width, height)
    particle.collide(p)
    sleep(0.02)

Canvas(height=200, width=400)

(0, 1)
1.2000000000000002


NameError: name 'bump' is not defined

In [9]:
A=np.array([[1,0],[0,0]])
print(A.dot(np.array([3,3])))

[3 0]
